In [ ]:
%run ../../v4/misc/data_access.py

In [ ]:
import os
from gensim.models.doc2vec import Doc2Vec,TaggedDocument
import textacy
from tqdm import tqdm, trange
import json

We are using spacy to kick out stop words, get the word lemmas and detect entities. Entity recognition is a bit funky.
We are using textacy for lazyness. Otherwise we might speed up things with a custom spacy pipeline.

Since building a doc2vec model requires to run over the text twice (once for building the dictionary and once for the model building) , we store the TaggedDocuments in a file (temporary).
- StreamTaggedDocumentobject: parses and stores the TaggedDocuments in a json file
- StreamTaggedDocumentFile: rereads the TaggedDocuments for the model building

**StreamTaggedDocumentobject** allows to set named_entity_tags, which would keep named entities that Spacy finds to the tags of an document. However when getting the results of similarity search, it would sometimes give one of these tags instead of the index, which fucks everything up

In [ ]:
base_folder = get_data_folder() + 'NAIL_DATAFIELD_txt/parsed_v3/merged/'

def name_to_file_name(author_name, file_format = 'txt'):
    return (author_name + '.' + file_format).replace(",","_")

def get_author_file(author_name):
    return base_folder + name_to_file_name(author_name)

def sentence_to_clean_word_list(text,get_named_entities = False):
    s_doc = textacy.Doc(text,lang='en').spacy_doc
    for t in s_doc:
        clean_words = [t.lemma_ for t in s_doc if not (t.is_punct or t.is_stop or t.lemma_ == '' or t.is_space)]
    if not get_named_entities:
        return clean_words, _
    else:
        return clean_words, [entity.text for entity in s_doc.ents]
        
def get_similar(model, words, top_n):
    inferred_vector = model.infer_vector(words)
    sims = model.docvecs.most_similar([inferred_vector], topn=top_n)
    return sims

def author_model_path(author_name):
    return get_model_folder() + "doc2vec/"+ name_to_file_name(author_name,'doc2vec')

def save_model(model, mode_file_path):
    model.clear_sims()
    model.delete_temporary_training_data()
    model.save(mode_file_path)
    
def load_author_model(author_name):
    return Doc2Vec.load(author_model_path(author_name))
        
        
class StreamTaggedDocumentobject:
    
    def __init__(self, file_path, store_while_streaming = True, named_entity_tags = False):
        self.file_path = file_path
        self.store_while_streaming = store_while_streaming
        self.temp_tagged_docs_file_path = self.file_path + '.taggeddocs.json'
        self.named_entity_tags = named_entity_tags
    
    def __iter__(self):
        if self.store_while_streaming:
            temp_file = open(self.temp_tagged_docs_file_path,'w')
        for uid, line in enumerate(open(self.file_path)):
            words, entities = sentence_to_clean_word_list(line, self.named_entity_tags)
            tags = [uid]
            if self.named_entity_tags:
                tags += entities
            tagged_doc = TaggedDocument(words=words, tags=tags)
            if self.store_while_streaming:                    
                temp_file.write(json.dumps({"words":tagged_doc.words, "tags":tagged_doc.tags}) + '\n')
            yield tagged_doc


class StreamTaggedDocumentFile:

    def __init__(self,tagged_doc_file_path):
        self.file_path = tagged_doc_file_path
        
    def __iter__(self):
        with open(self.file_path) as fin:
            for line in fin:
                td = json.loads(line)
                yield TaggedDocument(words = td["words"], tags = td["tags"])


Create a  model from an arbitrary text file or an ECO v4 Author based model with one call

In [ ]:
def create_model(corpus_file_path, model_file_path):
    model = Doc2Vec(size=100,  min_count=2, iter=55)
    tagged_docs_stream = StreamTaggedDocumentobject(corpus_file_path)
    print("building model from",corpus_file_path)
    print("building vocab...")
    model.build_vocab(tagged_docs_stream)
    tagged_docs_stream = StreamTaggedDocumentFile(tagged_docs_stream.temp_tagged_docs_file_path)
    print("training model...")
    model.train(tagged_docs_stream, total_examples=model.corpus_count, epochs=model.iter)
    print(model.estimate_memory())
    print("saving model to", model_file_path)
    save_model(model, model_file_path)
    print('done')
    
    
def create_author_model(author_name):
    create_model(get_author_file(author_name),author_model_path(author_name))

In [ ]:
create_author_model("Chomsky,Noam")

Now we create 2 classes to work with models, so to load them and to grab the associated text document.
- Doc2VecSimilarities: is for arbitrary doc2vec models 
- AuthorDoc2VecSimilarities for Authors (simpler call)

In [ ]:
class Doc2VecSimilarities:
    
    def __init__(self,model_file_path, text_file_path):
        print("Loading model")
        self.model = Doc2Vec.load(model_file_path)
        print("Reading corpus")
        self.text = list(textacy.fileio.read.read_file_lines(text_file_path))
        
    def get_top_n(self, sentence, top_n = 10):
        words = sentence_to_clean_word_list(sentence)[0]
        results = get_similar(self.model, words, top_n)
        return [self.text[result[0]].strip() for result in results]
    
    def get_all_more_similar_then(self, sentence, similarity = 0.8, max_results = 10):
        words = sentence_to_clean_word_list(sentence)[0]
        results = get_similar(self.model, words, max_results)
        return [self.text[result[0]].strip() for result in results if result[1] > similarity]        
        
class AuthorDoc2VecSimilarities(Doc2VecSimilarities):

    def __init__(self,author_name):
        Doc2VecSimilarities.__init__(self, author_model_path(author_name),get_author_file(author_name))

In [ ]:
sim = AuthorDoc2VecSimilarities("Chomsky,Noam")

In [ ]:
print(json.dumps(sim.get_top_n("West germany is the shit. literally it is shit", 10),indent=2))
print(json.dumps(sim.get_all_more_similar_then("fact is freedom in Nicaragua is a fraud .", 0.55),indent=2))

*More to come...*